In [1]:
from expand_folders import expand_folders
import os

In [7]:
import pandas as pd
df = pd.read_excel('/mnt/brics_data/datasets/Rio/fiocruz/raw/report_28_01_2024.xlsx')

In [8]:
df.head()

,Codigo MS,conf bact,crit diag
0,74,S,BAC
1,83,S,BAC
2,89,S,BAC
3,91,S,BAC
4,103,S,BAC


In [30]:
d= {'dataset_name':[], 'project_id':[], 'image_path':[], 'target':[], 'metadata':[], 'insertion_date':[]}

#,dataset_name,project_id,image_path,insertion_date,metadata,target

for path in expand_folders("/mnt/brics_data/datasets/Rio/fiocruz/raw/images/*.jpg"):
    #print(path)
    project_id = path.split('/')[-1].replace('.jpg','')
    id_number = project_id.split('_')[1]

    result = df.loc[df["Codigo MS"]==int(id_number)]['crit diag'].values
    if len(result)==0:
        target=0
    elif result[0]=='BAC':
        target=1
    else:
        target=0
    #print(id_number + "  ->  "+result.values[0])

    d['dataset_name'].append('fiocruz')
    d['project_id'].append(project_id)
    d['image_path'].append('images'+path.split('images')[-1])
    d['target'].append( target )
    d['metadata'].append({})
    d['insertion_date'].append('')

d=pd.DataFrame(d)

In [32]:
d.head()

,dataset_name,project_id,image_path,target,metadata,insertion_date
0,fiocruz,MS_1023_26.03,images/MS_1023_26.03.jpg,0,{},
1,fiocruz,MS_1033_26.03,images/MS_1033_26.03.jpg,0,{},
2,fiocruz,MS_725_26.03,images/MS_725_26.03.jpg,0,{},
3,fiocruz,MS_1026_26.03,images/MS_1026_26.03.jpg,0,{},
4,fiocruz,MS_1018_02.04,images/MS_1018_02.04.jpg,1,{},


In [33]:
d.to_csv("Rio_fiocruz_table_from_raw.csv")

In [36]:
d.loc[d.target==0].shape

(417, 6)

In [37]:
import pickle
import numpy as np
from itertools import compress

from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

def stratified_train_val_test_splits(df, n_folds,seed=512):
    cv_index = {'train_val': [], 'test': []}
    cv_train_test = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    sorts_train_test = []
    for train_val_idx, test_idx in cv_train_test.split(df.values, df.target.values):
        cv_index['test'].append(test_idx)
        cv_index['train_val'].append(train_val_idx)
    fold_idx = set(np.arange(0, n_folds, 1))
    for fold in fold_idx:
        val_idx = list(fold_idx - set([fold]))
        print('bins selected for val: ' + str(val_idx))
        sorts = []
        for i in tqdm(val_idx):
            l_val_idx = cv_index['test'][i]
            flt = ~pd.Series(cv_index['train_val'][fold]).isin(l_val_idx).values
            l_train_idx = np.array(list(compress(cv_index['train_val'][fold], flt)))
            sorts.append((l_train_idx, l_val_idx, cv_index['test'][fold]))
        sorts_train_test.append(sorts)
    return sorts_train_test

with open('splits.pic','wb') as f:
    splits = stratified_train_val_test_splits(d,10,512)
    pickle.dump(splits,f)

bins selected for val: [1, 2, 3, 4, 5, 6, 7, 8, 9]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 5716.04it/s]


bins selected for val: [0, 2, 3, 4, 5, 6, 7, 8, 9]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 6500.56it/s]


bins selected for val: [0, 1, 3, 4, 5, 6, 7, 8, 9]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 6021.49it/s]


bins selected for val: [0, 1, 2, 3, 4, 5, 6, 7, 9]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 9674.20it/s]


bins selected for val: [0, 1, 2, 3, 4, 5, 6, 7, 8]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 10161.17it/s]
